# Import packages:

In [1]:
from openbiolink.obl2021 import OBL2021Dataset
import torch.nn as nn
from torch.utils.data import DataLoader,TensorDataset
import math

D:\python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Accessing Data:

In [2]:
obl_dataset = OBL2021Dataset()

Dataset found in D:\programming repo-1\university of glasgow\msc project\MSc.Project\obl2021, omitting download...


In [3]:
# get the training, validation and test sets

obl_train_dataset = obl_dataset.training
obl_valid_dataset = obl_dataset.validation
obl_test_dataset = obl_dataset.testing

In [4]:
#checking out the tensor shape
print('The shape of the training tensor is : ', obl_train_dataset.shape)

#checking out the tensor dtype
print('The the data type of the training tensor is : ', obl_train_dataset.dtype)

The shape of the training tensor is :  torch.Size([4192002, 3])
The the data type of the training tensor is :  torch.int64


# Embedding Models:

## TransE:

In [5]:
class TransE(nn.Module):
    def __init__(self, device, num_entity, num_relation, emb_dim, gamma):
        super(TransE, self).__init__()
        self.device = device
        self.emb_dim = emb_dim
        self.num_entity = num_entity
        self.num_relation = num_relation
        
        #initialize entity embeddings
        self.entity_emb = self.initialize_emb(self.num_entity, self.emb_dim)

        #initialie relation embeddings
        self.relation_emb = self.initialize_emb(self.num_relation, self.emb_dim)
        self.relation_emb.weight.data.div_(self.relation_emb.weight.data.norm(p=2, dim=1, keepdim=True))
        #create the loss function
        loss_fn = nn.MarginRankingLoss(margin=gamma)
        

    def initialize_emb(self, num_emb, emb_dim):
        emb_weight_range = 6 / math.sqrt(emb_dim)
        emb = nn.Embedding(num_embeddings=num_emb, embedding_dim=emb_dim)
        emb.weight.data.uniform_( -emb_weight_range, emb_weight_range )
        return emb
    
    def forward(self, pos_triplet, neg_triplet):
        self.entity_emb.weight.data.div_(self.entity_emb.weight.data.norm(p=2, dim=1, keepdim=True))
        pos_distance = cal_distance(pos_triplet)
        neg_distance = cal_distance(neg_triplet)
        return loss_fn(pos_distance, neg_distance, torch.tensor([-1], dtype=torch.int64, device=self.device))
    
    def cal_distance(self, triplet):
        head = triplet[:,0]
        relation = triplet[:,1]
        tail = triplet[:,2]
        return (self.entity_emb(head) + self.relation(relation) - self.entity_emb(tail))
       
        

In [6]:
transE_model = TransE(device = 'cuda', num_entity = 100, num_relation = 10, emb_dim = 100, gamma = 0.1)

In [ ]:
def train_transE():
    for i in range(epoch):
        for index, batch_data in enumerate():